In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
datas = pd.concat([train, test], ignore_index = True)
print(train.head())
print(datas.head())

In [ ]:
print(train.info())
print(train.describe())